In [11]:
import os
import sys
import json
from dotenv import load_dotenv
from anthropic import Anthropic
from anthropic import __version__ as sdk_version

In [12]:
# Load API key
load_dotenv()
api_key = os.getenv("ANTHROPIC_API_KEY")
if not api_key:
    sys.exit("⚠️ Missing ANTHROPIC_API_KEY in .env")

In [13]:
# Initialize client
client = anthropic.Client(api_key=api_key)

In [14]:
# 3) List available models & pick the first Haiku variant :contentReference[oaicite:1]{index=1}
print(f"SDK version: {sdk_version}")
models_resp = client.models.list()     # GET /v1/models
haiku_models = [m.id for m in models_resp.data if "haiku" in m.id.lower()]
if not haiku_models:
    sys.exit("⚠️ No Haiku model found in Anthropic’s API.")
model_id = haiku_models[0]
print("Using Haiku model:", model_id)

SDK version: 0.52.2
Using Haiku model: claude-3-5-haiku-20241022


In [16]:
# 4) Simple completion (Messages API)
print("\n=== Simple Completion ===")
resp = client.messages.create(
    model=model_id,
    messages=[{"role": "user", "content": "Tell me a fun fact about space."}],
    max_tokens=80,
    temperature=0.7
)


=== Simple Completion ===


In [17]:
# resp.content is a list of content blocks with .type and .text
simple_text = "".join(
    block.text for block in resp.content if block.type == "text"
)
print(simple_text)

Did you know that on Venus, a day is longer than a year? Venus rotates so slowly that it takes 243 Earth days to complete one rotation on its axis, while it only takes 225 Earth days to complete its orbit around the Sun. This means a single day on Venus is actually longer than its year!


In [19]:
# 5) Streaming response
print("\n=== Streaming Response ===")
with client.messages.stream(
    model=model_id,
    messages=[{"role": "user", "content": "Write a 4-line poem about code."}],
    max_tokens=60,
    temperature=0.5
) as stream:
    for text in stream.text_stream:  # incremental text chunks
        print(text, end="", flush=True)
    print()


=== Streaming Response ===
Here's a 4-line poem about code:

Lines of logic, dancing light,
Syntax weaving patterns bright,
Algorithms solving puzzles deep,
Where human thought and machine meet.
